In [1]:
!pip install CoolProp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 17.1 MB/s eta 0:00:00


In [5]:
import CoolProp.CoolProp as ct

In [6]:
fluido = 'propane'

In [24]:
# m_dot = 300 # Tm/h
m_dot = 300 * 1e3 / 3600 # kg/s

In [18]:
# p1 = 100 # bar
p1 = 100 * 1e5 # Pa

In [20]:
# T1 = 540 # ºC1 =
T = 540 + 273.15

In [21]:
# p2 = 20 # bar
p2 = 20 * 1e5 # Pa

In [22]:
# T2 = 320 # ºC
T2 = 320 + 273.15 # K

In [28]:
h1 = ct.PropsSI('H','P',p1,'T',T1,fluido)

In [29]:
h2 = ct.PropsSI('H','P',p2,'T',T2,fluido)

In [26]:
print(f'Entalpía en 1 {h1:.1f} J/kg.')

Entalpía en 1 1073730.0 J/kg.


In [32]:
print(f'Entalpía en 1 {h2:.1f} J/kg.')

Entalpía en 1 1298442.8 J/kg.


In [36]:
Wcompresion = m_dot * (h1-h2)

In [37]:
print(f'Traballo de compresión: {Wcompresion / 1e6:.0f} J/kg')

Traballo de compresión: -19 J/kg


Unha bomba de calor que utiliza o refrixerante R-22 funciona cun caudal másico de 100 kg/h. O fluído entra no compresor a T1 = 300 K e P1 = 1 bar. Despreciase a a perda de calor do compresor. A presión de saída do compresor é de 5 bar. Se a eficiencia isentrópica do compresor é de 0,7 e a mecánica de 0,9, cal é a potencia absorbida do compresor e como é o refrixerante cando sae do compresor?

O libro de texto usa o Peng-Robinson EOS, así que para comparar, utilízao tamén.

In [39]:
!pip install thermo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.3 MB/s eta 0:00:00


In [54]:
# Estblecemos condicións e imports
from scipy.constants import bar, hour
from thermo import ChemicalConstantsPackage, PRMIX, CEOSLiquid, CEOSGas, FlashPureVLS

fluido = 'propane'

constantes, correlacions = ChemicalConstantsPackage.from_IDs([fluido])

T1 = 300.0 # K
P1 = 1 * bar # bar
P2 = 5 * bar
eta_isentropica = 0.7
eta_mecanica = 0.9

In [45]:
# Usamos os modelos de Peng-Robinson publicados orixinalmente (sen modificar)
eos_kwargs = dict(Tcs=constants.Tcs, Pcs=constants.Pcs, omegas=constants.omegas)
liquid = CEOSLiquid(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
gas = CEOSGas(PRMIX, HeatCapacityGases=correlations.HeatCapacityGases, eos_kwargs=eos_kwargs)
flasher = FlashPureVLS(constants, correlations, gas=gas, liquids=[liquid], solids=[])

In [50]:
# Flash ás condicións de entrada para obter enthalpñia inicial
estado_1 = flasher.flash(T=T1, P=P1)
# # Flash ás condicións de saída para obter enthalpía final. A entropía se conserva nos compresores e expansores!
estado_2_ideal = flasher.flash(S=estado_1.S(), P=P2)
# Calculamos o cambio de entalpía
delta_H_ideal = (estado_2_ideal.H()-estado_1.H())

# A definición de eficiencia isentrópica significa que a cantidadde real de calor aportado é:
# dH_real = dH_ideal/eta_isentropica
H_engadido_o_fluido_real = delta_H_ideal/eta_isentropica
print(f'Calor engadido ó fluido (real) = {H_engadido_o_fluido_real:.2f} J/kg')

Calor engadido ó fluido (real) = 6305.26 J/kg


In [56]:
estado_2 = flasher.flash(H=estado_1.H() + H_engadido_o_fluido_real, P=P2)

In [60]:
# To calculafr a potencia real, é mais convinnte usar a enthalpía másica
potencia_real_por_kg = (estado_2.H_mass() - estado_1.H_mass())/(eta_mecanica) # W/kg
potencia_real = potencia_real_por_kg * 100 / hour
print(f'A potencia real é {potencia_real:.0f} W')
print(f'A temperatura de saída é {estado_2.T: .2f} K')

A potencia real é 2251 W
A temperatura de saída é  405.98 K
